<a href="https://colab.research.google.com/github/duarte3333/Machine_Learning/blob/main/task1_imageanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading data
X = np.load('/content/drive/MyDrive/ML/Lab3/Xtrain_Classification1.npy') #6254 x 2352. 2352 = pixels x pixels x 3 (RGB)
y = np.load('/content/drive/MyDrive/ML/Lab3/ytrain_Classification1.npy') #6254. 1D vector
X_t = np.load('/content/drive/MyDrive/ML/Lab3/Xtest_Classification1.npy') #1764 x 2352. Has data from two distinct sources

FileNotFoundError: ignored

In [ ]:
#Data pre processing
Xs = (X).astype('float32')/np.max(X) #Scale down data [0,1]
Xs_test = (X_t).astype('float32')/np.max(X)

#Encoding
train_labels = keras.utils.to_categorical(y,num_classes=2) #One-hot enconding: 6254 x 2 matrix that is 0 or 1 depending on class
train_images = np.reshape(Xs, (len(Xs[:,0]),28,28,3)) #4D array. number of images x width x height x color
test_images = np.reshape(Xs_test, (len(Xs_test[:,0]), 28, 28, 3))

#Shuffling
permutation = np.random.permutation(len(Xs[:,0]))
train_images_s = train_images[permutation] #No shuffling done for test images
train_labels_s = train_labels[permutation]

In [ ]:
#Data split
X_train, X_validation, y_train, y_validation = train_test_split(train_images_s, train_labels_s, test_size=0.2)
print(X_train.shape)
print(y_train.shape)

In [ ]:
def plot_images(number_images,X_train):
  '''Plot a user defined number of images from X_train. Only really works for 10 images'''

  number_images = 10

  plt.figure(figsize=(12, 6))
  for i in range(number_images):
      plt.subplot(2, 5, i + 1)
      plt.imshow(X_train[i])
      plt.title(f"Image {i+1}")
      plt.axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
plot_images(10, X_train)

# First approach: MLP with imbalanced data

In [ ]:
#Building MLP neural network, Sequential (linear stack of Layers)
#kernel size 3 by 3 is normal in image data
#padding keeps images in the same size

#poolsize and strides -> Check notes on notion on maxpooling

MLP = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation = 'relu', padding= 'same', input_shape = (28,28,3)),
    MaxPool2D(pool_size =(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation = 'relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax')
])

#Flatten is always needed before using fully connected layer (Dense) for output


In [ ]:
MLP.summary() #See created neural network

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 28, 28, 32)        896       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 14, 14, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 7, 7, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_5 (Flatten)         (None, 3136)              0         
                                                                 
 dense_5 (Dense)             (None, 2)                

In [ ]:
#Compile MLP model
MLP.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

NOTE: Try 'SparseCategoricalCrossEntropy' as loss function.

In [ ]:
#Train MLP model
history = MLP.fit(x=X_train, y=y_train, validation_data=(X_validation, y_validation), batch_size=512, epochs=200, shuffle = True, verbose=2)

KeyboardInterrupt: ignored

In [ ]:
#Plot loss evolution in each epoch for training and validation set
plt.figure(1)
plt.clf()
plt.plot(history.history['loss'], label='train'),
plt.plot(history.history['val_loss'], label='train'), plt.show()

**Right now we have a have an overfitted neural network.**




In [ ]:
#Make predictions
predictions = MLP.predict(x = test_images, batch_size = 200, verbose=0) #Returns probability of each class
rounded_predictions= np.argmax(predictions, axis=-1) #Rounds it to 0 or 1

In [ ]:
#Visualize imbalance in training data
cont_0 = 0
for i in rounded_predictions:
  if i == 0:
    cont_0 += 1
print(cont_0, len(test_images[:,0])-cont_0) #Big difference between classes due to imbalanced training set

# Second approach: MLP with balanced data

In [ ]:
#Create ways to do data augmentation
gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                          height_shift_range=0.1, shear_range=0.15,
                         zoom_range=0.1, channel_shift_range=10,
                         horizontal_flip=True, vertical_flip = True)